In [ ]:
import prettytable
import tensorflow as tf
from keras import callbacks, initializers, optimizers, regularizers
from keras.layers import (LSTM, Bidirectional, Concatenate, Conv1D, Dense,
                          Dropout, Flatten, Input, MaxPooling1D,
                          TimeDistributed)
from keras.models import Model, Sequential
from sklearn.metrics import (accuracy_score, f1_score, hamming_loss,
                             precision_recall_curve, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from keras.models import load_model


In [ ]:
tf.__version__

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
# Loading Tensorboard Extension
%load_ext tensorboard

In [ ]:
from MultiChannelBiLSTMCNN import BalanceNet, time_logger_save

In [ ]:
# Metrics Calculator Function
def evaluate_model(real, predicted):
    accuracy = accuracy_score(real, predicted)
    hamLoss = hamming_loss(real, predicted)
    # element wise correctness
    term_wise_accuracy = np.sum(np.logical_not(
        np.logical_xor(real, predicted)))/real.size

    macro_precision = precision_score(real, predicted, average='macro')
    macro_recall = recall_score(real, predicted, average='macro')
    macro_f1 = f1_score(real, predicted, average='macro')

    micro_precision = precision_score(real, predicted, average='micro')
    micro_recall = recall_score(real, predicted, average='micro')
    micro_f1 = f1_score(real, predicted, average='micro')

    metricTable = prettytable.PrettyTable()
    metricTable.field_names = ["Metric", "Macro Value", "Micro Value"]
    metricTable.add_row(["Hamming Loss", "{0:.3f}".format(hamLoss), ""])
    metricTable.add_row(
        ["Term Wise Accuracy", "{0:.3f}".format(term_wise_accuracy), ""])

    metricTable.add_row(["Accuracy", "{0:.3f}".format(accuracy), ""])
    metricTable.add_row(["Precision", "{0:.3f}".format(
        macro_precision), "{0:.3f}".format(micro_precision)])
    metricTable.add_row(["Recall", "{0:.3f}".format(
        macro_recall), "{0:.3f}".format(micro_recall)])
    metricTable.add_row(
        ["F1-measure", "{0:.3f}".format(macro_f1), "{0:.3f}".format(micro_f1)])

    print(metricTable)

In [ ]:
#Helper Functions
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
  
def initial_boost(epoch):
    if epoch==0: return float(8.0)
    elif epoch==1: return float(4.0)
    elif epoch==2: return float(2.0)
    elif epoch==3: return float(1.5)
    else: return float(1.0)

def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.0001
        if epoch%33==0:multiplier = 10
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        #print("Epoch",epoch+1,"- learning_rate",rate)
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)


Loading Data

In [ ]:
mpstDF= pd.read_csv("mpst.csv")
mpstDF

In [ ]:
# Data Split Function
def get_partition_Embeddings(x_t1,x_t2,y,df,partition_nm):
    _df=df[df["split"]==partition_nm]
    index_list=list(_df.index)
    temp_array_x_t1=[]
    temp_array_x_t2=[]
    temp_array_y=[]
    for index in index_list:
        temp_array_x_t1.append(x_t1[index,:])
        temp_array_x_t2.append(x_t2[index,:])
        temp_array_y.append(y[index,:])
    temp_array_x_t1=np.array(temp_array_x_t1)
    temp_array_x_t2=np.array(temp_array_x_t2)
    temp_array_y=np.array(temp_array_y)
    return temp_array_x_t1,temp_array_x_t2, temp_array_y

In [ ]:
# LOADING BERT  EMBEDDINGS
bert_embedding=np.load(r"D:\CodeRepo\Thesis\Thesis\XLNet\xl_embeddings.npz")
# LOADING XLnet EMBEDDINGS
xl_embedding=np.load(r"D:\CodeRepo\Thesis\Thesis\XLNet\xl_embeddings.npz")
# LOADING LABELS Y
label_values=np.load(r"D:\CodeRepo\Thesis\Thesis\EDA\Y.npz")

In [ ]:
# BERT EMBEDDINGS T1 & T2
type1_BERT_Embeddings=bert_embedding["t1"]
type2_BERT_Embeddings=bert_embedding["t2"]
# XLNet EMBEDDINGS T1 & T2
type1_XL_Embeddings=xl_embedding["t1"]
type2_XL_Embeddings=xl_embedding["t2"]
# LABLES Y
label_values=label_values["arr_0"]

In [ ]:
# BERT

# FOR TRAIN
type1_BERT_Embeddings_Train,type2_BERT_Embeddings_Train,BERT_label_values_Train=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"train")
# FOR VALIDATION
type1_BERT_Embeddings_Val,type2_BERT_Embeddings_Val,BERT_label_values_Val=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"val")
# FOR TEST
type1_BERT_Embeddings_Test,type2_BERT_Embeddings_Test,BERT_label_values_Test=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"test")

In [ ]:
# XLNET

# FOR TRAIN
type1_XL_Embeddings_Train,type2_XL_Embeddings_Train,XLNET_label_values_Train=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"train")
# FOR VALIDATION
type1_XL_Embeddings_Val,type2_XL_Embeddings_Val,XLNET_label_values_Val=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"val")
# FOR TEST
type1_XL_Embeddings_Test,type2_XL_Embeddings_Test,XLNET_label_values_Test=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"test")

HAVING A LOOK AT THE SHAPES OF EACH PARTITION CREATED:

In [ ]:
print("SHAPES OF EACH PARTITION _ BERT\n")
print("type1_BERT_Embeddings_Train.shape: ", type1_BERT_Embeddings_Train.shape)
print("type2_BERT_Embeddings_Train.shape: ", type2_BERT_Embeddings_Train.shape)
print("BERT_label_values_Train.shape: "    , BERT_label_values_Train.shape)

print("type1_BERT_Embeddings_Val.shape: ", type1_BERT_Embeddings_Val.shape)
print("type2_BERT_Embeddings_Val.shape: ", type2_BERT_Embeddings_Val.shape)
print("BERT_label_values_Val.shape: "    , BERT_label_values_Val.shape)

print("type1_BERT_Embeddings_Test.shape: ", type1_BERT_Embeddings_Test.shape)
print("type2_BERT_Embeddings_Test.shape: ", type2_BERT_Embeddings_Test.shape)
print("BERT_label_values_Test.shape: "    , BERT_label_values_Test.shape)

print("SHAPES OF EACH PARTITION _ XLNET\n")

print("type1_XL_Embeddings_Train.shape: ", type1_XL_Embeddings_Train.shape)
print("type2_XL_Embeddings_Train.shape: ", type2_XL_Embeddings_Train.shape)
print("XLNET_label_values_Train.shape: " , XLNET_label_values_Train.shape)

print("type1_XL_Embeddings_Val.shape: ", type1_XL_Embeddings_Val.shape)
print("type2_XL_Embeddings_Val.shape: ", type2_XL_Embeddings_Val.shape)
print("XLNET_label_values_Val.shape: " , XLNET_label_values_Val.shape)

print("type1_XL_Embeddings_Test.shape: ", type1_XL_Embeddings_Test.shape)
print("type2_XL_Embeddings_Test.shape: ", type2_XL_Embeddings_Test.shape)
print("XLNET_label_values_Test.shape: " , XLNET_label_values_Test.shape)

print("SHAPES OF EACH PARTITION _ BERT\n")
print("type1_BERT_Embeddings_Train.: ", type1_BERT_Embeddings_Train)
print("type2_BERT_Embeddings_Train.: ", type2_BERT_Embeddings_Train)
print("BERT_label_values_Train.: "    , BERT_label_values_Train)

print("type1_BERT_Embeddings_Val.: ", type1_BERT_Embeddings_Val)
print("type2_BERT_Embeddings_Val.: ", type2_BERT_Embeddings_Val)
print("BERT_label_values_Val.: "    , BERT_label_values_Val)

print("type1_BERT_Embeddings_Test.shape: ", type1_BERT_Embeddings_Test)
print("type2_BERT_Embeddings_Test.shape: ", type2_BERT_Embeddings_Test)
print("BERT_label_values_Test.shape: "    , BERT_label_values_Test)

print("SHAPES OF EACH PARTITION _ XLNET\n")

print("type1_XL_Embeddings_Train: ", type1_XL_Embeddings_Train)
print("type2_XL_Embeddings_Train: ", type2_XL_Embeddings_Train)
print("XLNET_label_values_Train: " , XLNET_label_values_Train)

print("type1_XL_Embeddings_Val: ", type1_XL_Embeddings_Val)
print("type2_XL_Embeddings_Val: ", type2_XL_Embeddings_Val)
print("XLNET_label_values_Val: " , XLNET_label_values_Val)

print("type1_XL_Embeddings_Test: ", type1_XL_Embeddings_Test)
print("type2_XL_Embeddings_Test: ", type2_XL_Embeddings_Test)
print("XLNET_label_values_Test: " , XLNET_label_values_Test)


In [ ]:
# function to reshape my input
def tensor_reshape(input, timestep, features):
  reshaped= input.reshape(type1_BERT_Embeddings_Train.shape[0], timestep, features)
  return reshaped

STARTING THE MODEL
possible combination in inputs:
(timestep, features)
- (1, 768)
- (2, 384)
- (3, 256)
- (4, 192)
- (6, 128)
- (12, 64)

## MODEL (4, 192):


In [ ]:
INPUT_SHAPE =(4, 192)
EM_L_F_UNITS= 192
EM_L_T_UNITS= 192
# LEFT CHANNEL
LSTM_1F_UNITS= 128
LSTM_1T_UNITS= 128

CONV_2_FILTER= 24
CONV_2_KERNEL= 2
CONV_3_FILTER= 24
CONV_3_KERNEL= 2
CONV_5_FILTER= 24
CONV_5_KERNEL= 2
CONV_6_FILTER= 24
CONV_6_KERNEL= 2
CONV_8_FILTER= 24
CONV_8_KERNEL= 2

# RIGHT CHANNEL 
CONV_4F_FILTERS = 12
CONV_4F_KERNEL = 1
CONV_4T_FILTERS = 12
CONV_4T_KERNEL = 1

CONV_3F_FILTERS = 12
CONV_3F_KERNEL = 1
CONV_3T_FILTERS = 12
CONV_3T_KERNEL = 1

CONV_2F_FILTERS = 12
CONV_2F_KERNEL = 1
CONV_2T_FILTERS = 12
CONV_2T_KERNEL = 1

LSTM_2_C_L_UNITS = 12

OUTPUT_DENSE_UNIT =128
OUTPUT_SIZE =71



In [ ]:
optimizer_list = ['adam']

dataset_X={
    "bert_t1":[
        type1_BERT_Embeddings_Train,
        type1_BERT_Embeddings_Val,
        type1_BERT_Embeddings_Test
        ],
    "bert_t2":[
        type2_BERT_Embeddings_Train,
        type2_BERT_Embeddings_Val,
        type2_BERT_Embeddings_Test
        ],
    "xlnet_t1":[
        type1_XL_Embeddings_Train,
        type1_XL_Embeddings_Val,
        type1_XL_Embeddings_Test
        ],
    "xlnet_t2":[
        type2_XL_Embeddings_Train,
        type2_XL_Embeddings_Val,
        type2_XL_Embeddings_Test
    ]
}
dataset_Y ={
    "bert":[
        BERT_label_values_Train,
        BERT_label_values_Val,
        BERT_label_values_Test
        ],
    "xlnet":[
        XLNET_label_values_Train,
        XLNET_label_values_Val,
        XLNET_label_values_Test
        ]
}

inp_shape_str = "4_192"


In [ ]:
model_dict ={}
t_list =[x/10 for x in range(1,10)]
time_dict=[]

for ds in dataset_X.keys():
    for opt in optimizer_list:
        print("\nFOR OPTIMIZER: ",opt)
        key= "model_"+inp_shape_str+"_"+opt+"_"+ds
        model_type = ds+"_"+opt
        print("\n###############\nKEY= "+key+"\n#############")
        print("\n###############\nmodel_type= "+model_type+"\n#############")
        model_dict[key] = BalanceNet(INPUT_SHAPE,
                                    EM_L_F_UNITS,
                                    EM_L_T_UNITS,
                                    LSTM_1F_UNITS,
                                    LSTM_1T_UNITS,
                                    CONV_2_FILTER,
                                    CONV_2_KERNEL,
                                    CONV_3_FILTER,
                                    CONV_3_KERNEL,
                                    CONV_5_FILTER,
                                    CONV_5_KERNEL,
                                    CONV_6_FILTER,
                                    CONV_6_KERNEL,
                                    CONV_8_FILTER,
                                    CONV_8_KERNEL,
                                    CONV_4F_FILTERS,
                                    CONV_4F_KERNEL,
                                    CONV_4T_FILTERS,
                                    CONV_4T_KERNEL,
                                    CONV_3F_FILTERS,
                                    CONV_3F_KERNEL,
                                    CONV_3T_FILTERS,
                                    CONV_3T_KERNEL,
                                    CONV_2F_FILTERS,
                                    CONV_2F_KERNEL,
                                    CONV_2T_FILTERS,
                                    CONV_2T_KERNEL,
                                    LSTM_2_C_L_UNITS,
                                    OUTPUT_DENSE_UNIT,
                                    OUTPUT_SIZE,
                                    optimizer_name= opt)
        if 'bert' in ds:
            em_type='bert'
        if 'xlnet' in ds:
            em_type='xlnet'
        model_dict[key].fitModel(train_x =dataset_X[ds][0], 
                                            train_y=dataset_Y[em_type][0], 
                                            val_x= dataset_X[ds][1], 
                                            val_y = dataset_Y[em_type][1], 
                                            model_type = model_type, 
                                            epochs_count = 20 , 
                                            batch_count = 64,  time_dict=time_dict, key=key)
        model_dict[key].predictModel(test_x=dataset_X[ds][2], test_y=dataset_Y[em_type][2], threshold_list=t_list,  time_dict=time_dict, key=key)
time_logger_save(time_dict,inp_shape_str+"_time")


# END

In [ ]:
model_dict={}
model_dict["model_1_768_adam_bert_t1"]= BalanceNet(INPUT_SHAPE,
                                                    EM_L_F_UNITS,
                                                    EM_L_T_UNITS,
                                                    LSTM_1F_UNITS,
                                                    LSTM_1T_UNITS,
                                                    CONV_2_FILTER,
                                                    CONV_2_KERNEL,
                                                    CONV_3_FILTER,
                                                    CONV_3_KERNEL,
                                                    CONV_5_FILTER,
                                                    CONV_5_KERNEL,
                                                    CONV_6_FILTER,
                                                    CONV_6_KERNEL,
                                                    CONV_8_FILTER,
                                                    CONV_8_KERNEL,
                                                    CONV_4F_FILTERS,
                                                    CONV_4F_KERNEL,
                                                    CONV_4T_FILTERS,
                                                    CONV_4T_KERNEL,
                                                    CONV_3F_FILTERS,
                                                    CONV_3F_KERNEL,
                                                    CONV_3T_FILTERS,
                                                    CONV_3T_KERNEL,
                                                    CONV_2F_FILTERS,
                                                    CONV_2F_KERNEL,
                                                    CONV_2T_FILTERS,
                                                    CONV_2T_KERNEL,
                                                    LSTM_2_C_L_UNITS,
                                                    OUTPUT_DENSE_UNIT,
                                                    OUTPUT_SIZE,
                                                    optimizer_name= 'adam')

In [ ]:
model_dict["model_1_768_adam_bert_t1"].fitModel(train_x =dataset_X["bert_t1"][0], 
                                                train_y=dataset_Y['bert'][0], 
                                                val_x= dataset_X["bert_t1"][1], 
                                                val_y = dataset_Y['bert'][1], 
                                                model_type = "bert_t1_adam", 
                                                epochs_count = 20 , 
                                                batch_count = 64)